## Faster intersection between a hyperrectangular set and an axis-aligned halfspace #1800


- https://github.com/JuliaReach/LazySets.jl/issues/1800

In [1]:
using Revise, LazySets, BenchmarkTools, StaticArrays

┌ Info: Precompiling LazySets [b4f0291d-fe17-52bc-9479-3d1a343d9043]
└ @ Base loading.jl:1278


In [13]:
H = rand(Hyperrectangle)

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-0.3412273406525914, -0.511324484110687], [0.6860678348031224, 2.1614143501749137])

In [14]:
G = HalfSpace([1.0, 0.0], 0.0)

HalfSpace{Float64,Array{Float64,1}}([1.0, 0.0], 0.0)

In [15]:
Gs = HalfSpace(SingleEntryVector(1, 2, 1.0), 0.0)

HalfSpace{Float64,SingleEntryVector{Float64}}([1.0, 0.0], 0.0)

In [4]:
@btime intersection($G, $H)

  55.856 μs (299 allocations: 23.34 KiB)


HPolytope{Float64,Array{Float64,1}}(HalfSpace{Float64,Array{Float64,1}}[HalfSpace{Float64,Array{Float64,1}}([1.0, 0.0], -0.2294435195476123), HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0], -0.46442130050505004), HalfSpace{Float64,Array{Float64,1}}([-1.0, 0.0], 0.5843407933477632), HalfSpace{Float64,Array{Float64,1}}([0.0, -1.0], 0.5116373283465727)])

In [5]:
Q = Hyperrectangle(low=[-10., -10.], high=[0., 10]);

@btime intersection($Q, $H)

  127.377 ns (5 allocations: 416 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-0.4068921564476878, -0.4880293144258114], [0.17744863690007548, 0.02360801392076134])

## New method

In [6]:
B = rand(Hyperrectangle, dim=10)

high(B, 4)

2.9223898714343677

In [7]:
B = rand(BallInf, dim=10)
@time high(B, 4)

  0.004618 seconds (4.81 k allocations: 297.468 KiB)


1.5766675476693899

In [19]:
using LazySets: _intersection_interval_halfspace
using LazySets.Arrays

# intersection between an axis-aligned half-space and a hyperrectangular set
function _intersection(B::AbstractHyperrectangle, H::HalfSpace{N, <:SingleEntryVector{N}}) where {N}
    n = dim(H)
    a = H.a
    b = H.b
    i = a.i
    ai = a.v

    v_high_i = high(B, i)
    v_low_i = low(B, i)
    
    # intersect the half-space with the hyperrectangle's interval side
    (empty, lo, hi) = _intersection_interval_halfspace(v_low_i, v_high_i, ai, b, N)

    if empty
        return EmptySet{N}(n)

    else
        v_low′ = copy(low(B))
        v_low′[i] = lo

        v_high′ = copy(high(B))
        v_high′[i] = hi
        
        return Hyperrectangle(low=v_low′, high=v_high′)
    end
 end

# symmetric method
_intersection(H::HalfSpace{N, <:SingleEntryVector{N}}, B::AbstractHyperrectangle) where {N} = _intersection(B, H)

_intersection (generic function with 2 methods)

In [20]:
Gs

HalfSpace{Float64,SingleEntryVector{Float64}}([1.0, 0.0], 0.0)

In [21]:
H

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-0.3412273406525914, -0.511324484110687], [0.6860678348031224, 2.1614143501749137])

In [26]:
@btime _intersection($Gs, $H)

  244.244 ns (8 allocations: 624 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-0.5136475877278569, -0.5113244841106869], [0.5136475877278569, 2.1614143501749137])

In [27]:
@btime intersection($Gs, $H)

  52.012 μs (300 allocations: 23.56 KiB)


HPolytope{Float64,Array{Float64,1}}(HalfSpace{Float64,Array{Float64,1}}[HalfSpace{Float64,Array{Float64,1}}([1.0, 0.0], 0.0), HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0], 1.6500898660642267), HalfSpace{Float64,Array{Float64,1}}([-1.0, 0.0], 1.0272951754557138), HalfSpace{Float64,Array{Float64,1}}([0.0, -1.0], 2.6727388342856004)])

In [29]:
V = intersection(Gs, H)
@btime box_approximation($V)

  38.065 μs (213 allocations: 15.25 KiB)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-0.5136475877278569, -0.5113244841106869], [0.5136475877278569, 2.1614143501749137])

- total (master) `90us`

- new: `~250ns`